# Финальная работа по курсу Machine Learning Engineer

In [1]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-2.1.4-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 495.5 kB/s eta 0:04:13
   ---------------------------------------- 0.1/124.9 MB 939.4 kB/s eta 0:02:13
   ---------------------------------------- 0.2/124.9 MB 1.2 MB/s eta 0:01:46
   ---------------------------------------- 0.3/124.9 MB 1.2 MB/s eta 0:01:43
   ---------------------------------------- 0.3/124.9 MB 1.2 MB/s eta 0:01:42
   ---------------------------------------- 0.4/124.9 MB 1.4 MB/s eta 0:01:29
   ---------------------------------------- 0.6/124.9 MB 1.6 MB/s eta 0:01:17
   ---------------------------------------- 0.6/124.9 MB 1.6 MB/s eta 0:01:19
   ---------------------------------------- 0.7/124.9 MB 1.6 MB/s eta 0:01:18
   -

In [4]:
%pip install dill

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import  numpy as np
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import dill
import datetime
import os
from lightgbm import LGBMClassifier

## Data Understanding, Data Preparation и  Feature Engineering

In [2]:
#
df_target = pd.read_csv("train_data/train_target.csv")
df_target.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [3]:
# Загрузим данные из файлов в датафрейм...
# а также  все фичи развернем в столбцы, так как многочисленные эксперименты показали, что иначе т ребуемой точности не добиться...

ohe = OneHotEncoder(sparse_output=False)
df_final=pd.DataFrame()
for i in range(0, 12):
    file_path = "train_data/train_data_"+str(i)+".pq"
    df = dd.read_parquet(file_path)     
    df = df.compute()
    print("Количество пропусков в файле ", file_path," ", df.columns.isna().sum())
    all_features = df.columns.to_list()
    all_features.remove("id")
    encoded_data  = ohe.fit_transform(df[all_features])
    encoded_data = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out())
    #присоединим к датафрейму закодированные фичи
    df = pd.concat([df, encoded_data], axis=1) 
    #удалим все исходные фичи
    df = df.drop(all_features, axis=1)
    #сгруппируем по id клиента
    df_agg = df.groupby('id').sum()   
    #смержим с таргетом
    df_agg_new = pd.merge(df_agg, df_target, on='id', how='inner')
    #присоединим полученный датафрейм к итоговому
    df_final =pd.concat( [df_agg_new, df_final], axis=0)
df_final.head()    

Количество пропусков в файле  train_data/train_data_0.pq   0
Количество пропусков в файле  train_data/train_data_1.pq   0
Количество пропусков в файле  train_data/train_data_2.pq   0
Количество пропусков в файле  train_data/train_data_3.pq   0
Количество пропусков в файле  train_data/train_data_4.pq   0
Количество пропусков в файле  train_data/train_data_5.pq   0
Количество пропусков в файле  train_data/train_data_6.pq   0
Количество пропусков в файле  train_data/train_data_7.pq   0
Количество пропусков в файле  train_data/train_data_8.pq   0
Количество пропусков в файле  train_data/train_data_9.pq   0
Количество пропусков в файле  train_data/train_data_10.pq   0
Количество пропусков в файле  train_data/train_data_11.pq   0


,id,rn_1,rn_2,rn_3,rn_4,rn_5,rn_6,rn_7,rn_8,rn_9,rn_10,rn_11,rn_12,rn_13,rn_14,rn_15,rn_16,rn_17,rn_18,rn_19,rn_20,rn_21,rn_22,rn_23,rn_24,rn_25,rn_26,rn_27,rn_28,rn_29,rn_30,rn_31,rn_32,rn_33,rn_34,rn_35,rn_36,rn_37,rn_38,rn_39,...,enc_loans_account_holder_type_0,enc_loans_account_holder_type_1,enc_loans_account_holder_type_2,enc_loans_account_holder_type_3,enc_loans_account_holder_type_4,enc_loans_account_holder_type_5,enc_loans_account_holder_type_6,enc_loans_credit_status_0,enc_loans_credit_status_1,enc_loans_credit_status_2,enc_loans_credit_status_3,enc_loans_credit_status_4,enc_loans_credit_status_5,enc_loans_credit_status_6,enc_loans_credit_type_0,enc_loans_credit_type_1,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1,flag,pre_since_confirmed_15,pre_loans5_10,pre_loans530_17,pre_loans3060_0,pre_loans3060_4,pre_loans6090_0,pre_loans3060_3,pre_loans_total_overdue_1,pre_loans_max_overdue_sum_0
0,2750000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,1.0,0.0,7.0,0.0,0.0,6.0,1.0,5.0,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2750001,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,10.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2750002,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2750003,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1.0,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2750004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,9.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Посмотрим на размер итогового датафрейма...
# Поскольку пропуски связаны только с привнесенными колонками, вернее их разным количеством для разных строк, то смело заполним их нулями
df_final.fillna(0, inplace=True)

In [6]:
for column in df_final.columns:
    if column!="id":
        df_final[column] = df_final[column].astype("int8")

In [7]:
df_final.shape

(3000000, 479)

In [8]:
df_final.columns

Index(['id', 'rn_1', 'rn_2', 'rn_3', 'rn_4', 'rn_5', 'rn_6', 'rn_7', 'rn_8',
       'rn_9',
       ...
       'flag', 'pre_since_confirmed_15', 'pre_loans5_10', 'pre_loans530_17',
       'pre_loans3060_0', 'pre_loans3060_4', 'pre_loans6090_0',
       'pre_loans3060_3', 'pre_loans_total_overdue_1',
       'pre_loans_max_overdue_sum_0'],
      dtype='object', length=479)

for column in df_final.columns:
    if(column!="id"):
        df_final[column]=df_final[column].astype('int32')

In [9]:
df_final.dtypes

id                             int64
rn_1                            int8
rn_2                            int8
rn_3                            int8
rn_4                            int8
                               ...  
pre_loans3060_4                 int8
pre_loans6090_0                 int8
pre_loans3060_3                 int8
pre_loans_total_overdue_1       int8
pre_loans_max_overdue_sum_0     int8
Length: 479, dtype: object

In [10]:
df_final.head()

,id,rn_1,rn_2,rn_3,rn_4,rn_5,rn_6,rn_7,rn_8,rn_9,rn_10,rn_11,rn_12,rn_13,rn_14,rn_15,rn_16,rn_17,rn_18,rn_19,rn_20,rn_21,rn_22,rn_23,rn_24,rn_25,rn_26,rn_27,rn_28,rn_29,rn_30,rn_31,rn_32,rn_33,rn_34,rn_35,rn_36,rn_37,rn_38,rn_39,...,enc_loans_account_holder_type_0,enc_loans_account_holder_type_1,enc_loans_account_holder_type_2,enc_loans_account_holder_type_3,enc_loans_account_holder_type_4,enc_loans_account_holder_type_5,enc_loans_account_holder_type_6,enc_loans_credit_status_0,enc_loans_credit_status_1,enc_loans_credit_status_2,enc_loans_credit_status_3,enc_loans_credit_status_4,enc_loans_credit_status_5,enc_loans_credit_status_6,enc_loans_credit_type_0,enc_loans_credit_type_1,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_3,pclose_flag_0,pclose_flag_1,fclose_flag_0,fclose_flag_1,flag,pre_since_confirmed_15,pre_loans5_10,pre_loans530_17,pre_loans3060_0,pre_loans3060_4,pre_loans6090_0,pre_loans3060_3,pre_loans_total_overdue_1,pre_loans_max_overdue_sum_0
0,2750000,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,7,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,3,3,0,0,1,0,7,0,0,6,1,5,2,0,0,0,0,0,0,0,0,0,0
1,2750001,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,3,7,0,0,0,2,0,0,3,5,0,0,0,0,10,0,0,10,0,10,0,0,0,0,0,0,0,0,0,0,0
2,2750002,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,1,0,0,0,0,2,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0
3,2750003,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,1,0,0,0,0,3,0,0,3,0,1,2,0,0,0,0,0,0,0,0,0,0
4,2750004,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,9,0,0,0,0,0,0,0,1,8,0,0,0,0,0,0,1,5,3,0,0,0,9,0,0,9,0,9,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Разделим выборку на тренировочную и тестовую в соотношении 80% к 20%
X_train,  X_test, y_train, y_test  = train_test_split(df_final.drop(["id", "flag"], axis=1), df_final["flag"], stratify=df_final["flag"], test_size=0.3)

In [12]:
# Инициализация и привязка StandardScaler к обучающим данным
scaler = StandardScaler()
#scaler.fit(X_train)

# Трансформация обучающих и тестовых данных
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [13]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_test_scaled = pd.DataFrame(X_test_scaled)

In [14]:
print("Размер тренировочной выборки ", X_train.shape)

Размер тренировочной выборки  (2100000, 477)


In [15]:
print("Размер тестовой выборки ", X_test.shape)

Размер тестовой выборки  (900000, 477)


## Modeling

### Логистическая регрессия 

In [16]:
LR = LogisticRegression(class_weight='balanced')
LR.fit(X_train_scaled, y_train )

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [17]:
X_train_scaled.shape

(2100000, 477)

In [18]:
y_test_predict_LR=LR.predict_proba(X_test_scaled)

In [19]:
y_test_predict_LR.size

1800000

In [20]:
rs = roc_auc_score(y_test, y_test_predict_LR[:,1])
print("Логистическая регрессия - ROC-AUC = ", rs)

Логистическая регрессия - ROC-AUC =  0.7373442874844862


### Случайный лес

In [21]:
model_rf =RandomForestClassifier(n_jobs=-1, class_weight='balanced')
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [22]:
y_test_predict_rf = model_rf.predict_proba(X_test)
rs_rf = roc_auc_score(y_test, y_test_predict_rf[:,1])
print("Случайный лес - ROC_AUC = ", rs_rf)

Случайный лес - ROC_AUC =  0.708688210590184


### Градиентный бустинг XGB

In [23]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_jobs=-1, class_weight='balanced')# Create an instance of the XGBClassifier

#xgb_classifier = (objective='binary:logistic')

In [24]:
model_xgb.fit(X_train, y_train)

[17:41:23] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, ...)

In [25]:
y_test_predict_xgb  = model_xgb.predict_proba(X_test)
rs_xgb  = roc_auc_score(y_test, y_test_predict_xgb[:,1])
print("Метод градиентного бустинга- ROC_AUC = ", rs_xgb)

Метод градиентного бустинга- ROC_AUC =  0.7569269387795201


### Градиентный бустинг LGBM

In [26]:

model_xgb =  LGBMClassifier(n_jobs=-1, class_weight='balanced')# Create an instance of the XGBClassifier

#xgb_classifier = (objective='binary:logistic')

In [27]:
model_xgb.fit(X_train, y_train)

LGBMClassifier(class_weight='balanced')

In [28]:
y_test_predict_xgb  = model_xgb.predict_proba(X_test)
rs_xgb  = roc_auc_score(y_test, y_test_predict_xgb[:,1])
print("Метод градиентного бустинга- ROC_AUC = ", rs_xgb)

Метод градиентного бустинга- ROC_AUC =  0.7583541971862322


In [55]:
rs_xgb = roc_auc_score(y_test, y_test_predict_xgb[:,1])
print("Метод градиентного бустинга- ROC_AUC = ", rs_xgb) 

Метод градиентного бустинга- ROC_AUC =  0.7590742155077468
